In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

def test_classifier(clf, X_data, y_target, cv):
  score = cross_validate(clf, X_data, y_target, cv=cv, return_train_score=True, return_estimator=True)
  estimator = score['estimator'][0]
  train_score = float(format(score['train_score'].mean(),".5f"))
  test_score = float(format(score['test_score'].mean(),".5f"))
  fit_time = float(format(score['fit_time'].mean(),".5f"))
  print("Estimator: " + str(estimator))
  print("Accuracy on training set: " + str(train_score))
  print("Accuracy on test set: " + str(test_score))
  print("Fit time: " + str(fit_time))
  return estimator, train_score, test_score, fit_time

In [16]:
record_estimatror = []
record_train_score = []
record_test_score = []
record_fit_time = []

In [17]:
def record(rd):
  record_estimatror.append(rd[0])
  record_train_score.append(rd[1])
  record_test_score.append(rd[2])
  record_fit_time.append(rd[3])

In [5]:
filename = "/content/drive/My Drive/Colab Notebooks/diabetes.csv"

In [6]:
df = pd.read_csv(filename)

In [ ]:
df1 = pd.read_csv(filename)

In [8]:
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [129]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [131]:
print(df.groupby('Outcome').size())

Outcome
0    500
1    268
dtype: int64


In [132]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [42]:
df.shape

(768, 9)

In [ ]:
#deal with missing value 0, change to NaN 
#when Insulin==0, but Outcom ==1, not change

In [11]:
for col in ('Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'Age'): 
  for row in range(df.shape[0]):
    if df[col][row] == 0:
      if col == 'Insulin' and df['Outcome'][row] == 1: pass
      else: df[col][row] = np.nan     

<ipython-input-11-8ad22c6dea6a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: df[col][row] = np.nan
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
df.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [11]:
# use KNNImputer
from sklearn.impute import KNNImputer
# using n_neighbors = 14, weights = uniform
knn_imputer = KNNImputer(n_neighbors=14, weights='uniform')
# fit and transform
imputation_npnd = knn_imputer.fit_transform(df)

# convert the transform result from numpy.ndarray to pandas dataframe
df_imputation = pd.DataFrame(imputation_npnd, columns=df.columns)


In [12]:
df_imputation

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6.0,148.0,72.0,35.000000,206.214286,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.000000,58.857143,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,28.357143,210.285714,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.000000,94.000000,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.000000,168.000000,43.1,2.288,33.0,1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.000000,180.000000,32.9,0.171,63.0,0.0
764,2.0,122.0,70.0,27.000000,159.357143,36.8,0.340,27.0,0.0
765,5.0,121.0,72.0,23.000000,112.000000,26.2,0.245,30.0,0.0
766,1.0,126.0,60.0,29.785714,174.571429,30.1,0.349,47.0,1.0


In [13]:
X = df1.drop('Outcome', axis=1)
y = df1['Outcome']

In [14]:
# define the data and targe
X1 = df_imputation.iloc[:,0:9]
y1 = df_imputation.iloc[:,-1]

In [17]:
#Decision Tree
tree = DecisionTreeClassifier()
result = test_classifier(tree,X,y,cv=5) 
record(result)

Estimator: DecisionTreeClassifier()
Accuracy on training set: 1.0
Accuracy on test set: 0.71494
Fit time: 0.00534


In [18]:
tree = DecisionTreeClassifier()
result = test_classifier(tree,X1,y1,cv=5) 
record(result)

Estimator: DecisionTreeClassifier()
Accuracy on training set: 1.0
Accuracy on test set: 1.0
Fit time: 0.00336


In [146]:
training_accuracy = [0]
test_accuracy = [0]

for i in range(1,31):    
    clf = DecisionTreeClassifier(max_depth=i)
    score = cross_validate(clf, X, y, cv=5, return_train_score=True, return_estimator=True)      
    training_accuracy.append(score['train_score'].mean())
    test_accuracy.append(score['test_score'].mean())

fig = go.Figure()    
fig.layout.xaxis.range = [1, 31]
fig.layout.yaxis.range = [0.6, 1.1]  

fig.add_trace(go.Scatter(y=training_accuracy, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(y=test_accuracy, name='test accuracy', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="DecisionTreeClassifier", 
                  xaxis_title="max_depth", yaxis_title="score",legend_title="Score:")                 
fig.show()

print(max(test_accuracy))

0.7513793396146337


In [147]:
test_classifier(DecisionTreeClassifier(max_depth=5), X, y, cv=5)

Estimator: DecisionTreeClassifier(max_depth=5)
Accuracy on training set: 0.83757
Accuracy on test set: 0.7527
Fit time: 0.0045


(DecisionTreeClassifier(max_depth=5), 0.83757, 0.7527, 0.0045)

In [149]:
test_classifier(DecisionTreeClassifier(max_depth=7), X1, y1, cv=5)

Estimator: DecisionTreeClassifier(max_depth=7)
Accuracy on training set: 1.0
Accuracy on test set: 1.0
Fit time: 0.0034


(DecisionTreeClassifier(max_depth=7), 1.0, 1.0, 0.0034)

In [150]:
#Gaussian Naive Bayes
gnb = GaussianNB()
result = test_classifier(gnb, X, y, cv=5)
record(result)

Estimator: GaussianNB()
Accuracy on training set: 0.76595
Accuracy on test set: 0.75134
Fit time: 0.00339


In [19]:
#Gaussian Naive Bayes
gnb = GaussianNB()
result = test_classifier(gnb, X1, y1, cv=5)
record(result)

Estimator: GaussianNB()
Accuracy on training set: 1.0
Accuracy on test set: 1.0
Fit time: 0.00356


In [78]:
from sklearn.model_selection import GridSearchCV
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=100)}
gnbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=5, n_jobs=-1)
gnbModel_grid.fit(X, y)
print("Gaussian Navie Bayes best parameter : " + str(gnbModel_grid.best_estimator_))
print("Gaussian Navie Bayes best score : " + str(gnbModel_grid.best_score_))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Gaussian Navie Bayes best parameter : GaussianNB(var_smoothing=8.111308307896873e-06)
Gaussian Navie Bayes best score : 0.7857189224277832


In [151]:
test_classifier(GaussianNB(var_smoothing=8.111308307896872e-05), X, y, cv=5)

Estimator: GaussianNB(var_smoothing=8.111308307896872e-05)
Accuracy on training set: 0.75977
Accuracy on test set: 0.75528
Fit time: 0.00332


(GaussianNB(var_smoothing=8.111308307896872e-05), 0.75977, 0.75528, 0.00332)

In [29]:
from sklearn.model_selection import GridSearchCV
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=100)}
gnbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=5, n_jobs=-1)
gnbModel_grid.fit(X1, y1)
print("Gaussian Navie Bayes best parameter : " + str(gnbModel_grid.best_estimator_))
print("Gaussian Navie Bayes best score : " + str(gnbModel_grid.best_score_))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Gaussian Navie Bayes best parameter : GaussianNB(var_smoothing=1e-06)
Gaussian Navie Bayes best score : 1.0


In [32]:
test_classifier(GaussianNB(var_smoothing=4.32876128083053e-06), X1, y1, cv=5)

Estimator: GaussianNB(var_smoothing=4.32876128083053e-06)
Accuracy on training set: 0.99642
Accuracy on test set: 0.9961
Fit time: 0.00375


(GaussianNB(var_smoothing=4.32876128083053e-06), 0.99642, 0.9961, 0.00375)

In [30]:
test_classifier(GaussianNB(var_smoothing=8.111308307896872e-05), X1, y1, cv=5)

Estimator: GaussianNB(var_smoothing=8.111308307896872e-05)
Accuracy on training set: 0.81966
Accuracy on test set: 0.81905
Fit time: 0.00588


(GaussianNB(var_smoothing=8.111308307896872e-05), 0.81966, 0.81905, 0.00588)

In [ ]:
#Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
mulnb = MultinomialNB()
test_classifier(mulnb, X, y, cv=5)

Estimator: MultinomialNB()
Accuracy on training set: 0.601
Accuracy on test set: 0.594
Fit time: 0.00273


(MultinomialNB(), 0.601, 0.594, 0.00273)

In [ ]:
#Complement Naive Bayes
from sklearn.naive_bayes import ComplementNB
cnb = ComplementNB()
test_classifier(cnb, X, y, cv=5)

Estimator: ComplementNB()
Accuracy on training set: 0.595
Accuracy on test set: 0.586
Fit time: 0.00239


(ComplementNB(), 0.595, 0.586, 0.00239)

In [ ]:
#Bernoulli Naive Bayes
from sklearn.naive_bayes import BernoulliNB 
bnb = BernoulliNB()
test_classifier(bnb, X, y, cv=5)

Estimator: BernoulliNB()
Accuracy on training set: 0.654
Accuracy on test set: 0.647
Fit time: 0.00272


(BernoulliNB(), 0.654, 0.647, 0.00272)

In [153]:
#Neural Networks
mlp = MLPClassifier()
result = test_classifier(mlp, X, y, cv=5)
record(result)

Estimator: MLPClassifier()
Accuracy on training set: 0.70409
Accuracy on test set: 0.67328
Fit time: 0.32137


In [80]:
tuned_parameters = {'hidden_layer_sizes': range(1,200),
                    'alpha':   [0.001, 0.01, 1, 2]}

model = GridSearchCV(MLPClassifier(max_iter=10000), tuned_parameters, cv=5)
model.fit(X, y)
print("Best MLP model: " + str(model.best_estimator_))
print("Best MLP score: " + str(model.best_score_))

Best MLP model: MLPClassifier(alpha=2, hidden_layer_sizes=150, max_iter=10000)
Best MLP score: 0.7450178513469653


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
tuned_parameters = {'hidden_layer_sizes': range(1,100),
                    'alpha':   [0.01, 1, 2]}

model = GridSearchCV(MLPClassifier(max_iter=1000), tuned_parameters, cv=5)
model.fit(X_scaled, y)
print("Best MLP model after Scaling: " + str(model.best_estimator_))
print("Best MLP score after Scaling: " + str(model.best_score_))

Best MLP model after Scaling: MLPClassifier(alpha=0.01, hidden_layer_sizes=80, max_iter=1000)
Best MLP score after Scaling: 0.7839062897886426


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
tuned_parameters = {'hidden_layer_sizes': range(100,200),
                    'alpha':   [0.01, 1, 2]}

model = GridSearchCV(MLPClassifier(max_iter=10000), tuned_parameters, cv=5)
model.fit(X_scaled, y)
print("Best MLP model after Scaling: " + str(model.best_estimator_))
print("Best MLP score after Scaling: " + str(model.best_score_))

Best MLP model after Scaling: MLPClassifier(alpha=1, hidden_layer_sizes=115, max_iter=10000)
Best MLP score after Scaling: 0.7864867158984807


In [ ]:
test_classifier(MLPClassifier(alpha=0.001, hidden_layer_sizes=121, max_iter=1000), X_scaled, y, cv=5)

Estimator: MLPClassifier(alpha=0.001, hidden_layer_sizes=121, max_iter=1000)
Accuracy on training set: 0.84115
Accuracy on test set: 0.76312
Fit time: 2.18632


(MLPClassifier(alpha=0.001, hidden_layer_sizes=121, max_iter=1000),
 0.84115,
 0.76312,
 2.18632)

In [37]:
test_classifier(MLPClassifier(alpha=0.001, hidden_layer_sizes=121, max_iter=1000), X1, y1, cv=5)

Estimator: MLPClassifier(alpha=0.001, hidden_layer_sizes=121, max_iter=1000)
Accuracy on training set: 0.9645
Accuracy on test set: 0.95324
Fit time: 0.82286


(MLPClassifier(alpha=0.001, hidden_layer_sizes=121, max_iter=1000),
 0.9645,
 0.95324,
 0.82286)

In [118]:
#KNN
knn = KNeighborsClassifier()
result = test_classifier(knn, X, y, cv=5)
record(result)

Estimator: KNeighborsClassifier()
Accuracy on training set: 0.88346
Accuracy on test set: 0.85158
Fit time: 0.00341


In [66]:
training_accuracy = [0]
test_accuracy = [0]

for i in range(1,31):    
    clf = KNeighborsClassifier(n_neighbors=i)
    score = cross_validate(clf, X, y, cv=5, return_train_score=True, return_estimator=True)      
    training_accuracy.append(score['train_score'].mean())
    test_accuracy.append(score['test_score'].mean())

fig = go.Figure()    
fig.layout.xaxis.range = [1, 31]
fig.layout.yaxis.range = [0.6, 1]  

fig.add_trace(go.Scatter(y=training_accuracy, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(y=test_accuracy, name='test accuracy', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="KNeighborsClassifier", 
                  xaxis_title="n_neighbors", yaxis_title="score",legend_title="Score:")                 
fig.show()

print(max(test_accuracy))

0.7487734487734488


In [ ]:
test_classifier(KNeighborsClassifier(n_neighbors=14), X, y, cv=5)

Estimator: KNeighborsClassifier(n_neighbors=14)
Accuracy on training set: 0.7819
Accuracy on test set: 0.75786
Fit time: 0.00383


(KNeighborsClassifier(n_neighbors=14), 0.7819, 0.75786, 0.00383)

In [16]:
training_accuracy = [0]
test_accuracy = [0]

for i in range(1,31):    
    clf = KNeighborsClassifier(n_neighbors=i)
    score = cross_validate(clf, X1, y1, cv=5, return_train_score=True, return_estimator=True)      
    training_accuracy.append(score['train_score'].mean())
    test_accuracy.append(score['test_score'].mean())

fig = go.Figure()    
fig.layout.xaxis.range = [1, 31]
fig.layout.yaxis.range = [0.6, 1]  

fig.add_trace(go.Scatter(y=training_accuracy, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(y=test_accuracy, name='test accuracy', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="KNeighborsClassifier", 
                  xaxis_title="n_neighbors", yaxis_title="score",legend_title="Score:")                 
fig.show()

print(max(test_accuracy))

0.8620066208301502


In [27]:
test_classifier(KNeighborsClassifier(n_neighbors=14), X1, y1, cv=5)

Estimator: KNeighborsClassifier(n_neighbors=14)
Accuracy on training set: 0.8737
Accuracy on test set: 0.86201
Fit time: 0.00436


(KNeighborsClassifier(n_neighbors=14), 0.8737, 0.86201, 0.00436)

In [18]:
#Support Vector Machine
svc = SVC()
result = test_classifier(svc, X, y, cv=5)
record(result)

Estimator: SVC()
Accuracy on training set: 0.77279
Accuracy on test set: 0.75915
Fit time: 0.01803


In [19]:
training_accuracy = [0]
test_accuracy = [0]

for i in range(1,31):    
    clf = SVC(C=i)
    score = cross_validate(clf, X, y, cv=5, return_train_score=True, return_estimator=True)      
    training_accuracy.append(score['train_score'].mean())
    test_accuracy.append(score['test_score'].mean())

fig = go.Figure()    
fig.layout.xaxis.range = [1, 31]
fig.layout.yaxis.range = [0.6, 1]  

fig.add_trace(go.Scatter(y=training_accuracy, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(y=test_accuracy, name='test accuracy', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="SVC", 
                  xaxis_title="C", yaxis_title="score",legend_title="Score:")                 
fig.show()

print(max(test_accuracy))

0.7643833290892115


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
training_accuracy = []
test_accuracy = []
training_accuracy.append(0)
test_accuracy.append(0)

for i in range(1,31):    
    clf = SVC(C=i)
    score = cross_validate(clf, X_scaled, y, cv=5, return_train_score=True, return_estimator=True)      
    training_accuracy.append(score['train_score'].mean())
    test_accuracy.append(score['test_score'].mean())

fig = go.Figure()    
fig.layout.xaxis.range = [1, 31]
fig.layout.yaxis.range = [0.6, 1]  

fig.add_trace(go.Scatter(y=training_accuracy, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(y=test_accuracy, name='test accuracy', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="SVC", 
                  xaxis_title="C", yaxis_title="score",legend_title="Score:")                 
fig.show()

print(max(test_accuracy))

0.7839232662762073


In [ ]:
svc = SVC(gamma='auto')
test_classifier(svc, X, y, cv=5)

Estimator: SVC(gamma='auto')
Accuracy on training set: 1.000
Accuracy on test set: 0.651
Fit time: 0.02912


(SVC(gamma='auto'), '1.000', '0.651', '0.02912')

In [ ]:
test_classifier(SVC(C=37), X_scaled, y, cv=5)

Estimator: SVC(C=37)
Accuracy on training set: 0.85417
Accuracy on test set: 0.79044
Fit time: 0.02407


(SVC(C=37), 0.85417, 0.79044, 0.02407)

In [22]:
training_accuracy = [0]
test_accuracy = [0]

for i in range(1,100):    
    clf = SVC(C=i)
    score = cross_validate(clf, X1, y1, cv=5, return_train_score=True, return_estimator=True)      
    training_accuracy.append(score['train_score'].mean())
    test_accuracy.append(score['test_score'].mean())

fig = go.Figure()    
fig.layout.xaxis.range = [1, 101]
fig.layout.yaxis.range = [0.6, 1]  

fig.add_trace(go.Scatter(y=training_accuracy, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(y=test_accuracy, name='test accuracy', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="SVC", 
                  xaxis_title="C", yaxis_title="score",legend_title="Score:")                 
fig.show()

print(max(test_accuracy))

0.8854256854256853


In [23]:
test_classifier(SVC(C=94), X1, y1, cv=5)

Estimator: SVC(C=94)
Accuracy on training set: 0.91471
Accuracy on test set: 0.88543
Fit time: 0.0326


(SVC(C=94), 0.91471, 0.88543, 0.0326)

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X1_scaled = scaler.fit_transform(X1)

In [26]:
test_classifier(SVC(C=94), X1_scaled, y1, cv=5)

Estimator: SVC(C=94)
Accuracy on training set: 1.0
Accuracy on test set: 1.0
Fit time: 0.0038


(SVC(C=94), 1.0, 1.0, 0.0038)

In [ ]:
print(record_estimatror)
print(record_train_score)
print(record_test_score)
print(record_fit_time)

[DecisionTreeClassifier(), GaussianNB(), MLPClassifier(), KNeighborsClassifier(), SVC()]
[1.0, 0.766, 0.724, 0.798, 0.773]
[0.702, 0.751, 0.69, 0.724, 0.759]
[0.00531, 0.00296, 0.31634, 0.00398, 0.01607]


In [ ]:
X_name = ['DecisionTreeClassifier', 'GaussianNB', 'MLPClassifier','KNeighborsClassifier','SVC' ]

In [ ]:
fig = go.Figure()    
#fig.layout.xaxis.range = [0, 30]
fig.layout.yaxis.range = [0, 1.1]  

fig.add_trace(go.Scatter(x=X_name, y=record_train_score, name='training accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(x=X_name,y=record_test_score, name='test accuracy', mode='lines+markers'))
fig.add_trace(go.Scatter(x=X_name,y=record_fit_time, name='fit time', mode='lines+markers'))

fig.update_layout(autosize=False, width=1000, height=500,
                  title="Performance of Classifiers with default Parameter", 
                  xaxis_title="Classifiers", yaxis_title="score")                 
fig.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipe = Pipeline([("classifier", DecisionTreeClassifier())])

search_space = [{"classifier": [DecisionTreeClassifier()],
                 "classifier__max_depth": np.arange(1,30),
                 "classifier__min_samples_leaf": np.arange(1,10)                  
                },             
                {"classifier": [GaussianNB()], 
                 "classifier__var_smoothing": np.logspace(0,-9, num=100)                                                 
                },          
                {"classifier": [MLPClassifier(max_iter=1000)],
                 "classifier__hidden_layer_sizes": np.arange(1,200),                 
                 "classifier__alpha": [0.001, 0.01, 1, 2]                  
                },
                {"classifier": [KNeighborsClassifier()],
                 "classifier__n_neighbors": np.arange(2,30), 
                 'classifier__weights': ['uniform', 'distance']                                  
                },
                {"classifier": [SVC()],
                 "classifier__C": np.arange(1,50),
                 "classifier__kernel": ['linear', 'rbf', 'poly'],
                 "classifier__gamma": ['scale', 'auto']                          
                }]

In [ ]:
model = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)
best_model = model.fit(X, y)
print("Best model: " + str(best_model.best_estimator_))
print("Best score: " + str(best_model.best_score_))

Best model: Pipeline(steps=[('classifier', SVC(C=4, kernel='linear'))])
Best score: 0.7708853238265003


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
model = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)
best_model = model.fit(X_scaled, y)
print("Best model after standard scaling: " + str(best_model.best_estimator_))
print("Best score after standard scaling: " + str(best_model.best_score_))

Best model after standard scaling: Pipeline(steps=[('classifier',
                 MLPClassifier(alpha=0.001, hidden_layer_sizes=121,
                               max_iter=1000))])
Best score after standard scaling: 0.7865376453611747


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
model = GridSearchCV(pipe, search_space, cv=5, verbose=0, n_jobs=-1)
best_model = model.fit(X_scaled, y)
print("Best model after MinMax scaling: " + str(best_model.best_estimator_))
print("Best score after MinMax scaling: " + str(best_model.best_score_))

Best model after MinMax scaling: Pipeline(steps=[('classifier', SVC(C=37))])
Best score after MinMax scaling: 0.7904422375010609


In [ ]:
best_model.cv_results_

{'mean_fit_time': array([1.47476196e-03, 2.43668556e-03, 1.52339935e-03, 1.18956566e-03,
        1.27677917e-03, 1.24163628e-03, 1.24368668e-03, 1.18703842e-03,
        1.99432373e-03, 2.35509872e-03, 2.56047249e-03, 1.46522522e-03,
        1.73263550e-03, 1.83911324e-03, 1.45745277e-03, 2.02922821e-03,
        1.49865150e-03, 1.42250061e-03, 3.18474770e-03, 2.44064331e-03,
        1.74498558e-03, 1.85041428e-03, 1.80902481e-03, 1.72119141e-03,
        2.29139328e-03, 5.84292412e-03, 1.98855400e-03, 2.01992989e-03,
        2.02584267e-03, 2.03166008e-03, 2.28438377e-03, 2.86264420e-03,
        1.98149681e-03, 2.04567909e-03, 2.02174187e-03, 1.98578835e-03,
        2.28443146e-03, 4.28414345e-03, 3.77750397e-03, 2.58545876e-03,
        2.23283768e-03, 2.27046013e-03, 2.25181580e-03, 2.18639374e-03,
        4.13861275e-03, 4.52933311e-03, 2.50420570e-03, 2.60338783e-03,
        2.47311592e-03, 2.48131752e-03, 6.20474815e-03, 5.44142723e-03,
        4.96339798e-03, 2.33755112e-03, 2.72879